# Julian McClellan
## Data Task
### Selection

First, I go to [CropScape](https://nassgeodata.gmu.edu/CropScape/) and select Autauga County, Alabama as my area of interest. ![](./img/select_al.png)

Then, I generate a .tif image from the application utilizing all available data in the Cropland Data Layer for this county.

![](./img/select_cdl.png)

![](./img/save_tif.png)

### Dataset creation

Browsing through stack exchange, I find a way to extract latitude and longitude from the .tif file I downloaded. The code below also extracts the "pixel values" of the image (integers that specify the colors indicating different land cover types). I'll decode these later using the [2017 AL metadata.](https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_al17.htm)

The package utilized below, rasterio, also has support for reading .img files, which the [national download CDL files](https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php) linked to from the CropScape application contain.

In [1]:
# non-original comments in this cell denoted with "##"
# https://gis.stackexchange.com/questions/129847/obtain-coordinates-and-corresponding-pixel-values-from-geotiff-using-python-gdal
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
fname = './D__CDL_NASS_DATA_CACHE_extract_1942934485_CDL_2017_01001.tif'

## Read raster
with rasterio.open(fname) as r:
    T0 = r.transform  ## upper-left pixel corner affine transform
    p1 = Proj(r.crs)
    A = r.read()  ## pixel values

## All rows and columns
cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

## Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
## Function to convert pixel row/column index (from 0) to easting/northing at centre
rc2en = lambda r, c: (c, r) * T1

## All eastings and northings (there is probably a faster way to do this)
eastings, northings = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

## Project all longitudes, latitudes
p2 = Proj(proj='latlong',datum='WGS84') # can use other standards besides wgs84
longs, lats = transform(p1, p2, eastings, northings)

C:\Users\jcm\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0


I made a csv *pixel_values_2017_dict_al.csv* using the [metadata .htm file for AL in 2017](https://www.nass.usda.gov/Research_and_Science/Cropland/metadata/metadata_al17.htm). I utilized [regexr.com](https://regexr.com/) but this could be done with Python's re package. This will be used to decode the pixel values contained in variable *A*.

![](./img/regexr.png)

In [2]:
import pandas as pd

pixel_val_dict_al = pd.read_csv('pixel_values_2017_dict_al.csv', names=['pixel_value_num', 'land_cover'])
pixel_val_dict_al

,pixel_value_num,land_cover
0,0,Background
1,1,Corn
2,2,Cotton
3,3,Rice
4,4,Sorghum
...,...,...
128,247,Turnips
129,248,Eggplants
130,249,Gourds
131,250,Cranberries


Aiming for a csv here with a header of *lat, lon, pixel_value_num, land_cover*, so I flatten each of the arrays, and stack along the appropriate axis and create a dataframe

In [3]:
final_df = pd.DataFrame(np.stack([lats.flatten(), longs.flatten(), A[0].flatten()], axis=1), columns=['lat', 'lon', 'pixel_value_num'])

Merge the metadata csv I created to decode the pixel values.

In [4]:
final_df = pd.merge(final_df, pixel_val_dict_al, how='left', on='pixel_value_num')
final_df

,lat,lon,pixel_value_num,land_cover
0,32.748283,-86.910921,0.0,Background
1,32.748258,-86.910601,0.0,Background
2,32.748232,-86.910280,0.0,Background
3,32.748206,-86.909960,0.0,Background
4,32.748181,-86.909639,0.0,Background
...,...,...,...,...
2678539,32.275695,-86.428682,0.0,Background
2678540,32.275668,-86.428364,0.0,Background
2678541,32.275641,-86.428045,0.0,Background
2678542,32.275614,-86.427727,0.0,Background


Check if the dictionary didn't cover any pixel_value_nums

In [5]:
final_df.isna().any()

lat                False
lon                False
pixel_value_num    False
land_cover         False
dtype: bool

### Final dataset

The column pixel_value_num seems to be a float unncessarily, make it an int. 

Additionally, I remove the background pixels (The black, useless pixels in the original .tif image)

In [6]:
final_df['pixel_value_num'] = final_df['pixel_value_num'].astype('int64')
final_df = final_df[final_df.land_cover != 'Background']
final_df

,lat,lon,pixel_value_num,land_cover
1549,32.707421,-86.414659,152,Shrubland
1550,32.707393,-86.414339,152,Shrubland
1551,32.707366,-86.414019,152,Shrubland
1552,32.707339,-86.413699,141,Deciduous Forest
1553,32.707312,-86.413378,141,Deciduous Forest
...,...,...,...,...
2675667,32.307787,-86.810985,111,Open Water
2675668,32.307761,-86.810666,111,Open Water
2675669,32.307735,-86.810347,111,Open Water
2675670,32.307709,-86.810028,111,Open Water


I sanity check the last row of this dataframe, which should be open water.

![](./img/sanity_check1.png)
![](./img/sanity_check2.png)

Yep.

In [7]:
# write it
final_df.to_csv('jcm_dataset.csv', index=False)

### Closing notes

So right now the data is about as spatially disaggregated as one can have it. Aggregating up again to zip codes, counties, states, etc. is indeed possible, and would probably be best done by retrieving the appropriate shapefile(s) from census data or an appropriate government source online, and checking whether the lat, lon coordinates fall within a certain shapefile.

Alternatively, one could also use the [Google Maps API](https://github.com/shugamoe/ucpd_reports/blob/master/coords.py) to aggregate up to zip codes, counties, and states, or create some sort of bot to utilize CropScape's area of interest button to assist in aggregation (bot specifies aggregation area, and the .tif file can be parsed with a column to acknowledge the .tif file's aggregation level).